# Exercise 3: Financial Statement Analysis
For this exercise, let’s perform some financial statement analysis on the car industry, which is associated with the SIC code of 3771. We will rank companies, for their fiscal years ending in 2022, on their (i) current ratio, (ii) debt to equity ratio, and (iii) return on equity. First, I’ll walk through how to obtain the current ratios and then ask you to use a similar procedure to identify which firm has the highest return on equity and which has the highest debt to equity. 

## Step 1: Import Required Stata Modules

In [ ]:
import os, re, sys, json, requests, getpass, urllib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from datetime import datetime
from urllib.parse import urlencode

## Step 2: Obtain XBRL API Access

In [ ]:
password = getpass.getpass(prompt = 'Enter Your XBRL US Password: ')

body_auth = {'username' : 'vac35@psu.edu', 
            'client_id': 'Obtain Client ID from XBRL Website', 
            'client_secret' : 'Obtain Client Secret from XBRL Website', 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb' }

payload = urlencode(body_auth)
url = 'https://api.xbrl.us/oauth2/token'
headers = {"Content-Type": "application/x-www-form-urlencoded"}

res = requests.request("POST", url, data = payload, headers = headers)
auth_json = res.json()

if 'error' in auth_json:
    print ("Access Denied")
else:
    print ("Access Granted.")
    
access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''

## Step 3: Query the XBRL API

In this step, you need to identify the tags associated with your request. For this example, we are interested in current assets and current liabilites. An easy way to approach this would be to go to an actual XBRL filing from a relevant company to see the tag associated with cash. 

*Please note that tags can and will change over time.*

In [ ]:
XBRL_Elements = ['AssetsCurrent',
                 'LiabilitiesCurrent']

##### Substep 3.2: Identify Other Relevant Parameters
In this step, you need to supply the years that you are requesting the data for and the specific filings from which the relevant data was filed on. For our example, we are only interested in annual data for 2022. We also need to specify which industry we are interested in.

*Please note that, as compared to Exercise 1, we are not entering any parameters for companies. This is because we are interested in all companies within the SIC code of 3711 for this query.*

In [ ]:
Filings = ['10-K']
Years = ['2022'] 
SIC = ['3711']

##### Substep 3.3: Run the Query

The query will store the data into a dataframe titled *df*.

Please note that the line *'entity.cik': ','.join(Companies),* has been removed from the parameters list and the line *'report.sic-code: ','.join(SIC)* is used instead.

In [ ]:
Fields = ['entity.cik',
          'entity.name.sort(ASC)',
          'report.filing-date',
          'period.fiscal-year',
          'report.document-type',
          'concept.local-name',
          'fact.value',
          'unit',
          'report.sic-code']

Parameters = {'concept.local-name': ','.join(XBRL_Elements),
              'period.fiscal-period': 'Y',
              'period.fiscal-year': ','.join(Years),
              'unit': 'USD',
              'report.document-type': ','.join(Filings),
              'report.sic-code': ','.join(SIC)}  

has_dimensions = 'FALSE'

if has_dimensions == 'ALL':
    dimension_options = ['TRUE', 'FALSE']
else:
    dimension_options = [has_dimensions]

search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'
    
all_res_list = []
for dimensions_param in dimension_options:

    print('Getting the data for: "fact.has-dimensions" = {}'.format(dimensions_param))
    
    done_retrieving_all_results = False
    offset = 0

    while not done_retrieving_all_results:

        Parameters['fact.has-dimensions'] = dimensions_param
        Parameters['fields'] = ','.join(Fields) + ',fact.offset({})'.format(offset) 

        res = requests.get(search_endpoint, params = Parameters, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        
        res_json = res.json()
        res_list = res_json['data']
        all_res_list += res_list
        
        paging_dict = res_json['paging']

        print('Number of Observations Obtained: ', paging_dict['count'])

        if paging_dict['count'] >= 2000:
            offset += paging_dict['count']
        else:
            done_retrieving_all_results = True
    
df = pd.DataFrame(all_res_list)
print('Number of Observations: {}'.format(len(df)))

## Step 4: Clean the Data

##### Substep 4.1: Keep Relevant Variables

In this step, we create a new dataframe titled *fundamentals* and then rename our variables into more manageable names.

In [ ]:
fundamentals = df[['entity.cik', 'entity.name', 'fact.value', 'report.filing-date', 'concept.local-name']]
fundamentals.columns = ['CIK', 'Company', 'value', 'filing', 'account']

##### Substep 4.2: Remove Duplicate Observations

We want to keep one observation per CIK - fiscal year. First, we sort the observations by CIK, filing date, and account. There are two options that are available here - restated data or as-filed data. If we want to see how the market responded to the filing, we keep the first observation using 'keep = 'first''. If we want to see the most accurate figure, use the last option using 'keep = 'last''. This will provide restated data if the data was restated. Please note that this may cause the results to change over time.

In [ ]:
fundamentals = fundamentals.sort_values(by = ['CIK', 'filing', 'account'])
fundamentals = fundamentals.drop_duplicates(subset = ['CIK', 'filing', 'account'], keep = 'last')

##### Substep 4.3: Reorganize Data 

In this step, we want to reorganize the data to one CIK - Fiscal Year observations. For each observation, the stable variables are *CIK* and *Company*. We need to pivot *account* and *value*. 

In [ ]:
fundamentals = fundamentals.pivot_table(index=["CIK", "Company"], columns="account", values="value", aggfunc='min')
fundamentals = fundamentals.rename_axis(None, axis=1)
fundamentals.reset_index(inplace=True)

##### Substep 4.3: Further Clean Data

First, rename the relevant variables (i.e., *CIK*, *Company*, *Current Assets*, and *Current Liabilities*). Next, we will create the *Current Ratio* variable. 

In [ ]:
fundamentals.columns = ['CIK', 'Company', 'Current Assets', 'Current Liabilities']
fundamentals['Current Ratio'] = fundamentals['Current Assets'] / fundamentals['Current Liabilities']

##### Substep 4.4: Further (Further) Clean Data

First, keep only relevant variables (i.e., *CIK*, *Company*, and *Current Ratio*). Next, we sort the file by *Current Ratio*.

In [ ]:
fundamentals = fundamentals[['CIK', 'Company', 'Current Ratio']]
fundamentals = fundamentals.sort_values(by = 'Current Ratio', ascending = False)

## Step 5: Display the Results

In [ ]:
fundamentals

## Token Refresher

In [ ]:
token = token if newrefresh != '' else refresh_token 

refresh_auth = {'client_id': 'a04fc50b-a62c-4e96-8578-6e71b3c9bc52', 
                'client_secret' : 'dc6805e2-f03b-4f68-808d-89cfffcfc469', 
                'grant_type' : 'refresh_token',
                'platform' : 'ipynb', 
                'refresh_token' : ''.join(token) }
refreshres = requests.post(url, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Token Refreshed')